<a href="https://colab.research.google.com/github/jahnavi-2116/AI-ML-projects/blob/main/Chroma_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.8 MB/s eta 

In [2]:
documents = [
    "The Taj Mahal is located in Agra, India. It was built by Mughal Emperor Shah Jahan.",
    "Python is a popular programming language for AI and data science.",
    "The solar system has eight planets revolving around the sun.",
]

In [3]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
embeddings = embedding_model.encode(documents).tolist()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [5]:
import chromadb
from chromadb.config import Settings

# Use the new client configuration
client = chromadb.PersistentClient(path="./chroma_db")

collection = client.get_or_create_collection(name="rag_demo")

# Add data
for i, (doc, emb) in enumerate(zip(documents, embeddings)):
    collection.add(
        documents=[doc],
        embeddings=[emb],
        ids=[str(i)]
    )

print("Stored documents in vector DB.")

Stored documents in vector DB.


In [6]:
query = "Where is the Taj Mahal?"
query_embedding = embedding_model.encode(query).tolist()

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=1
)

print("\nQuery Results:")
for i, result in enumerate(results['documents'][0]):
    print(f"{i+1}. {result}")


Query Results:
1. The Taj Mahal is located in Agra, India. It was built by Mughal Emperor Shah Jahan.


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [7]:
count = collection.count()
print(f"Number of items in the collection: {count}")

Number of items in the collection: 3


In [8]:
retrieved_items = collection.get(
    ids=["0", "1"], # Example: retrieve items with ids "0" and "1"
    include=['embeddings', 'documents', 'metadatas'] # Specify what to include in the results
)

print("\nRetrieved Items:")
for i, item_id in enumerate(retrieved_items['ids']):
    print(f"Item ID: {item_id}")
    print(f"Document: {retrieved_items['documents'][i]}")
    # print(f"Embedding: {retrieved_items['embeddings'][i]}") # Uncomment to see embeddings
    # print(f"Metadata: {retrieved_items['metadatas'][i]}") # Uncomment to see metadatas
    print("-" * 20)


Retrieved Items:
Item ID: 0
Document: The Taj Mahal is located in Agra, India. It was built by Mughal Emperor Shah Jahan.
--------------------
Item ID: 1
Document: Python is a popular programming language for AI and data science.
--------------------


In [9]:
from google.colab import files

uploaded_files = files.upload()

Saving Reflections on Project Management - Example Paper(1).pdf to Reflections on Project Management - Example Paper(1).pdf


In [10]:
!pip install -q PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 81.1 MB/s eta 0:00:00


In [11]:
import fitz  # PyMuPDF

extracted_texts = {}
for filename, content in uploaded_files.items():
    try:
        # Save the uploaded file temporarily to process it with PyMuPDF
        with open(filename, 'wb') as f:
            f.write(content)

        # Open the PDF file
        doc = fitz.open(filename)
        text = ""
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text()

        extracted_texts[filename] = text
        print(f"Processed file: {filename}")

    except Exception as e:
        print(f"Error processing file {filename}: {e}")

print("\nExtracted Texts:")
for filename, text in extracted_texts.items():
    print(f"--- {filename} ---")
    print(text[:200] + "..." if len(text) > 200 else text) # Print first 200 characters
    print("-" * 20)

Processed file: Reflections on Project Management - Example Paper(1).pdf

Extracted Texts:
--- Reflections on Project Management - Example Paper(1).pdf ---
  
  
  
  
  
   and...
--------------------


In [14]:
new_embeddings = {}
for filename, text in extracted_texts.items():
    try:
        embedding = embedding_model.encode(text).tolist()
        new_embeddings[filename] = embedding
        print(f"Generated embedding for: {filename}")
    except Exception as e:
        print(f"Error generating embedding for {filename}: {e}")

print("\nGenerated Embeddings (first 5 elements):")
for filename, embedding in new_embeddings.items():
    print(f"--- {filename} ---")
    print(embedding[:5])
    print("-" * 20)

Generated embedding for: Reflections on Project Management - Example Paper(1).pdf

Generated Embeddings (first 5 elements):
--- Reflections on Project Management - Example Paper(1).pdf ---
[-0.012968317605555058, 0.056039806455373764, 0.05007002130150795, 0.02069755271077156, 0.01320011168718338]
--------------------


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [15]:
try:
    for filename, text in extracted_texts.items():
        embedding = new_embeddings.get(filename)
        if embedding:
            collection.add(
                documents=[text],
                embeddings=[embedding],
                ids=[filename]  # Using filename as the ID
            )
            print(f"Added document '{filename}' to ChromaDB.")
        else:
            print(f"Skipping '{filename}' as embedding was not generated.")

    print("\nSuccessfully added new documents to ChromaDB.")

except Exception as e:
    print(f"Error adding documents to ChromaDB: {e}")

Added document 'Reflections on Project Management - Example Paper(1).pdf' to ChromaDB.

Successfully added new documents to ChromaDB.


In [16]:
print(f"Number of items in the collection: {count}")

# Retrieve some items from the collection
# You can specify ids to retrieve specific items, or use the limit and offset parameters
# to retrieve a range of items.
# Including the ID of the newly uploaded document
retrieved_items = collection.get(
    ids=["0", "1", "Resume2025V2.pdf"], # Example: retrieve items with ids "0", "1" and the new document's filename
    include=['embeddings', 'documents', 'metadatas'] # Specify what to include in the results
)

print("\nRetrieved Items:")
for i, item_id in enumerate(retrieved_items['ids']):
    print(f"Item ID: {item_id}")
    print(f"Document: {retrieved_items['documents'][i][:200]}...") # Print first 200 characters of the document
    # print(f"Embedding: {retrieved_items['embeddings'][i]}") # Uncomment to see embeddings
    # print(f"Metadata: {retrieved_items['metadatas'][i]}") # Uncomment to see metadatas
    print("-" * 20)

Number of items in the collection: 3

Retrieved Items:
Item ID: 0
Document: The Taj Mahal is located in Agra, India. It was built by Mughal Emperor Shah Jahan....
--------------------
Item ID: 1
Document: Python is a popular programming language for AI and data science....
--------------------
